In [1]:
import os

from itertools import product

import xarray as xr
import dask
import cftime
import numpy as np
import pandas as pd

import util

PROJECT = "NCGD0011"
USER = os.environ["USER"]

In [2]:
basins = ['North_Atlantic_basin', 'North_Pacific_basin', 'South', 'Southern_Ocean']
npolygon = dict(
    North_Atlantic_basin=150,
    North_Pacific_basin=200,
    South=300,
    Southern_Ocean=40,
)

In [3]:
mths = ['-' + str(m).zfill(2) for m in range(1,13)]
yrs = np.array([str(yr).zfill(4) for yr in range(347, 363)])

timestamps = np.char.add(np.repeat(yrs, len(mths)),
                         np.tile(mths, len(yrs))
           )

timestamps

array(['0347-01', '0347-02', '0347-03', '0347-04', '0347-05', '0347-06',
       '0347-07', '0347-08', '0347-09', '0347-10', '0347-11', '0347-12',
       '0348-01', '0348-02', '0348-03', '0348-04', '0348-05', '0348-06',
       '0348-07', '0348-08', '0348-09', '0348-10', '0348-11', '0348-12',
       '0349-01', '0349-02', '0349-03', '0349-04', '0349-05', '0349-06',
       '0349-07', '0349-08', '0349-09', '0349-10', '0349-11', '0349-12',
       '0350-01', '0350-02', '0350-03', '0350-04', '0350-05', '0350-06',
       '0350-07', '0350-08', '0350-09', '0350-10', '0350-11', '0350-12',
       '0351-01', '0351-02', '0351-03', '0351-04', '0351-05', '0351-06',
       '0351-07', '0351-08', '0351-09', '0351-10', '0351-11', '0351-12',
       '0352-01', '0352-02', '0352-03', '0352-04', '0352-05', '0352-06',
       '0352-07', '0352-08', '0352-09', '0352-10', '0352-11', '0352-12',
       '0353-01', '0353-02', '0353-03', '0353-04', '0353-05', '0353-06',
       '0353-07', '0353-08', '0353-09', '0353-10', 

In [4]:
%%time
path = '/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Mengyang_Global_OAE_Experiments/archive/'

rows = []
offset = 0
for n, b in enumerate(basins):
    
    polygon_ids = [f'{i:03d}' for i in np.arange(offset, offset + npolygon[b])]    
    offset += npolygon[b]
    
    for i, p_id in enumerate(polygon_ids):
        
        for m in ['01', '04', '07', '10']:
            ndx = np.int32(m) - 1
            dates = timestamps[ndx:ndx + 180]
            
            case = f'smyle-fosi.{b}.alk-forcing-{b}.{i:03d}-1999-{m}'
            files = [f'{path}/{case}/ocn/hist/{case}.pop.h.{d}.nc' for d in dates]
            
            rows.append(
                dict(polygon=i, polygon_id=p_id, basin=b, start_date=dates[0], files=files)
            )

index_fields = ['polygon', 'basin', 'start_date']
df = pd.DataFrame(rows).set_index(index_fields)
df

CPU times: user 275 ms, sys: 48.3 ms, total: 324 ms
Wall time: 325 ms


polygon_id  \
polygon basin                start_date              
0       North_Atlantic_basin 0347-01           000   
                             0347-04           000   
                             0347-07           000   
                             0347-10           000   
1       North_Atlantic_basin 0347-01           001   
...                                            ...   
38      Southern_Ocean       0347-10           688   
39      Southern_Ocean       0347-01           689   
                             0347-04           689   
                             0347-07           689   
                             0347-10           689   

                                                                                     files  
polygon basin                start_date                                                     
0       North_Atlantic_basin 0347-01     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-04     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-07     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-10     [/glade/campaign/cesm/development/bgcwg/projec...  
1       North_Atlantic_basin 0347-01     [/glade/campaign/cesm/development/bgcwg/projec...  
...                                                                                    ...  
38      Southern_Ocean       0347-10     [/glade/campaign/cesm/development/bgcwg/projec...  
39      Southern_Ocean       0347-01     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-04     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-07     [/glade/campaign/cesm/development/bgcwg/projec...  
                             0347-10     [/glade/campaign/cesm/development/bgcwg/projec...  

[2760 rows x 2 columns]

In [5]:
start_dates = list(df.index.unique(level='start_date'))
polygons = [df.xs((b, start_dates[0]), level=('basin', 'start_date')).index[0] for b in basins]

In [6]:
fpath_smyle = '/glade/campaign/cesm/development/espwg/SMYLE/initial_conditions/SMYLE-FOSI/ocn/proc/tseries/month_1/'
ds_ctrl = xr.open_dataset(
    f'{fpath_smyle}/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.pop.h.ALK.030601-036812.nc', 
    decode_times=False,
)


tb_var = ds_ctrl.time.attrs["bounds"]
time_units = ds_ctrl.time.units
calendar = ds_ctrl.time.calendar

time = cftime.num2date(
    ds_ctrl[tb_var].mean('d2'),
    units=time_units,
    calendar=calendar,
)
yyyymm_ctrl = np.array([f'{t.year:04d}-{t.month:02d}' for t in time]) #.sel(timestamps)
tndx = np.arange(
    np.where(yyyymm_ctrl == timestamps[0])[0],
    np.where(yyyymm_ctrl == timestamps[-1])[0] + 1, 1,
)

yyyymm_ctrl = yyyymm_ctrl[tndx]

da_ctrl = ds_ctrl.ALK.isel(z_t=0, time=tndx)
da_ctrl

/glade/derecho/scratch/mclong/tmp/ipykernel_19156/4039259044.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tndx = np.arange(


<xarray.DataArray 'ALK' (time: 192, nlat: 384, nlon: 320)> Size: 94MB
[23592960 values with dtype=float32]
Coordinates:
    z_t      float32 4B 500.0
    ULONG    (nlat, nlon) float64 983kB ...
    ULAT     (nlat, nlon) float64 983kB ...
    TLONG    (nlat, nlon) float64 983kB ...
    TLAT     (nlat, nlon) float64 983kB ...
  * time     (time) float64 2kB 1.267e+05 1.267e+05 ... 1.325e+05 1.325e+05
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Alkalinity
    units:         meq/m^3
    grid_loc:      3111
    cell_methods:  time: mean

In [7]:
tndx_ctrl = {
    '0347-01': 0,
    '0347-04': 3,
    '0347-07': 6,
    '0347-10': 9,
}

In [8]:
cluster, client = util.get_ClusterClient(memory="2GB", project=PROJECT, walltime="12:00:00")
cluster.scale(256)
client

/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41557 instead
  warnings.warn(
/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project w

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/41557/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/41557/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.98:45931,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/41557/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
n = ds_ctrl.KMT.where(ds_ctrl.KMT > 0).count().values.item()
n

86096

In [10]:
%%time 

@dask.delayed
def comparison(index):
    """return RMSE for field compared to reference"""
    
    files = df.loc[index].files
    da_list = [xr.open_dataset(f)['ALK_ALT_CO2'].isel(time=0, z_t=0) for f in files]

    start_date = df.loc[index].name[-1]
    tndx0 = tndx_ctrl[start_date]

    rmse = []
    for i, da_test in enumerate(da_list):
        rmse_i = np.sqrt(np.nansum((da_test.values - da_ctrl.isel(time=i+tndx0).values) ** 2) / n)
        rmse.append(rmse_i.item())

    return np.array(rmse)
        

rmse = []
for b, d in product(basins, start_dates):
    # get the indexes for these polygons — keep the reference as the first index
    polygons = df.xs((b, d), level=('basin', 'start_date')).index
    print((b, d))

    objs_rmse = []
    for p in polygons:
        p_ndx = (p, b, d)
        polygon_id = df.loc[p_ndx].polygon_id
        objs_rmse.append(dict(polygon=p, basin=b, start_date=d, polygon_id=polygon_id, rmse=comparison(p_ndx)))
    
    rmse.extend(dask.compute(objs_rmse)[0])

('North_Atlantic_basin', '0347-01')


/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 126.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


('North_Atlantic_basin', '0347-04')


/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 126.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


('North_Atlantic_basin', '0347-07')


...

('Southern_Ocean', '0347-07')


/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 126.12 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


('Southern_Ocean', '0347-10')


/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 126.12 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 15min 50s, sys: 8min 34s, total: 24min 25s
Wall time: 56min 57s


In [11]:
df_comp = pd.DataFrame(rmse).set_index(index_fields)
df_comp

,,,polygon_id,rmse
polygon,basin,start_date,,
0,North_Atlantic_basin,0347-01,000,"[0.015219307708251998, 0.1023125586303381, 0.1..."
1,North_Atlantic_basin,0347-01,001,"[0.01211056091438915, 0.12287374184497717, 0.0..."
2,North_Atlantic_basin,0347-01,002,"[0.0121200128322259, 0.12400077929228648, 0.08..."
3,North_Atlantic_basin,0347-01,003,"[0.012957612270305062, 0.09339506483579374, 0...."
4,North_Atlantic_basin,0347-01,004,"[0.0225619112358421, 0.15670700573085475, 0.11..."
...,...,...,...,...
35,Southern_Ocean,0347-10,685,"[1.4155668414088693, 4.007592052027372, 3.5496..."
36,Southern_Ocean,0347-10,686,"[1.4152385331441366, 4.007386272096258, 3.5478..."
37,Southern_Ocean,0347-10,687,"[1.4151438923687336, 4.00839695582341, 3.54812..."


In [12]:
df_comp.to_pickle('comparison_data.pkl')

In [13]:
client.close()
cluster.close()

/glade/work/mclong/miniconda3/envs/mcdr-atlas/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
